In [1]:
import requests
import pandas as pd 
from pandas.io.json import json_normalize


url = "https://raw.githubusercontent.com/OBOFoundry/OBOFoundry.github.io/master/registry/ontologies.jsonld"
obo = requests.get(url)

In [2]:
obo_json = obo.json()['ontologies']

obo_df = json_normalize(obo_json)
obo_df.head()


obo_df_active = obo_df[obo_df['activity_status'] == 'active']



In [3]:
obo_df_active.columns

Index(['DO wiki', 'activity_status', 'alternatePrefix', 'alternativePrefix',
       'biosharing', 'browsers', 'build.checkout', 'build.email_cc',
       'build.infallible', 'build.insert_ontology_id', 'build.method',
       'build.notes', 'build.oort_args', 'build.path', 'build.source_url',
       'build.system', 'canonical', 'contact.contact', 'contact.email',
       'contact.github', 'contact.label', 'createdWith', 'dependencies',
       'depicted_by', 'description', 'development.id_policy', 'documentation',
       'domain', 'exampleClass', 'facebook', 'funded_by', 'google_plus',
       'homepage', 'id', 'in_foundry', 'in_foundry_order',
       'integration_server', 'is_obsolete', 'jobs', 'label', 'layout',
       'license.label', 'license.logo', 'license.url', 'mailing_list',
       'ontology_purl', 'page', 'preferredPrefix', 'products', 'publications',
       'redirects', 'releases', 'replaced_by', 'repository', 'source',
       'source_url', 'taxon.id', 'taxon.label', 'termgenie',

In [4]:


obo_df_active_selection = obo_df_active[['title', 'description', 'taxon.id','homepage']]
 

In [17]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_id(name):
    
    NAME_LOWERCASE = name.lower()
    endpoint_url = "https://query.wikidata.org/sparql"

    query = """SELECT ?item ?itemLabel
            WHERE { 
              ?item p:P31/ps:P31/wdt:P279* wd:Q324254.
              ?item rdfs:label ?itemLabel. 
              FILTER(CONTAINS(LCASE(?itemLabel),""" +'"' + NAME_LOWERCASE + '"' + """ @en)). 
            } limit 10
            """

    def get_results(endpoint_url, query):
        sparql = SPARQLWrapper(endpoint_url)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()


    results = get_results(endpoint_url, query)

    wikidata_id = results['results']['bindings'][0]['item']['value'].split('/')[4]
    
    return(wikidata_id)


ids = []


for index, row in obo_df_active_selection.iterrows():
    print(index)
    try:
        ids.append(get_id(row["title"].lower()))
    except:
        print("failed: ")
        print(row[0])
        ids.append("donotexist")  




0
1
2
failed: 
Human Disease Ontology
3
4
5
failed: 
Phenotype And Trait Ontology
6
7
failed: 
PRotein Ontology (PRO)
8
9
failed: 
Zebrafish anatomy and development ontology
10
failed: 
Anatomical Entity Ontology
11
failed: 
Agronomy Ontology
12
failed: 
Ascomycete phenotype ontology
13
14
failed: 
Antibiotic Resistance Ontology
15
failed: 
Beta Cell Genomics Ontology
16
17
failed: 
Biological Spatial Ontology
18
failed: 
BRENDA tissue / enzyme source
19
20
21
22
failed: 
CHEBI Integrated Role Ontology
23
24
25
26
failed: 
Clinical measurement ontology
27
28
failed: 
Ctenophore Ontology
29
30
failed: 
Dictyostelium discoideum anatomy
31
failed: 
Dictyostelium discoideum phenotype ontology
32
failed: 
Drug-drug Interaction and Drug-drug Interaction Evidence Ontology
33
failed: 
The Drug-Drug Interactions Ontology
34
failed: 
Drosophila Phenotype Ontology
35
failed: 
The Drug Ontology
36
failed: 
The Data Use Ontology
37
38
failed: 
An ontology of core ecological entities
39
failed: 
Env

In [18]:
obo_df_active_selection['wikidata_ids'] = ids

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
get_id('Xenopus Anatomy Ontology')

'Q55118650'

In [22]:
for index, row in obo_df_active_selection.iterrows():
        if row['wikidata_ids'] == 'donotexist' :
            print("CREATE\n" + 
                  'LAST|Len|' + '"' + row['title'] + '"\n' +
                  'LAST|Den|' + '"'+ row['description'] + '"\n' +
                  'LAST|P31|' + "Q81314568"  + '\n' +
                  'LAST|P856|' + '"' + row['homepage']  + '"' + '\n' +
                  "LAST|P361|" + "Q7072326" + '\n')
            
        else :
            wid = row['wikidata_ids']
                
            print( wid + '|P31|' + "Q81314568"  + '\n' +
            wid + '|P856|' + '"' + row['homepage']  + '"' + '\n' +
            wid + "|P361|" + "Q7072326" + '\n') 

Q4866972|P31|Q81314568
Q4866972|P856|"http://ifomis.org/bfo/"
Q4866972|P361|Q7072326

Q55118295|P31|Q81314568
Q55118295|P856|"http://www.ebi.ac.uk/chebi"
Q55118295|P361|Q7072326

CREATE
LAST|Len|"Human Disease Ontology"
LAST|Den|"An ontology for describing the classification of human diseases organized by etiology."
LAST|P31|Q81314568
LAST|P856|"http://www.disease-ontology.org"
LAST|P361|Q7072326

Q135085|P31|Q81314568
Q135085|P856|"http://geneontology.org/"
Q135085|P361|Q7072326

Q55118501|P31|Q81314568
Q55118501|P856|"http://obi-ontology.org"
Q55118501|P361|Q7072326

CREATE
LAST|Len|"Phenotype And Trait Ontology"
LAST|Den|"An ontology of phenotypic qualities (properties, attributes or characteristics)"
LAST|P31|Q81314568
LAST|P856|"https://github.com/pato-ontology/pato/"
LAST|P361|Q7072326

Q7201529|P31|Q81314568
Q7201529|P856|"http://browser.planteome.org/amigo"
Q7201529|P361|Q7072326

CREATE
LAST|Len|"PRotein Ontology (PRO)"
LAST|Den|"An ontological representation of protein-relate

TypeError: must be str, not float